In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats import zscore

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

df = pd.read_csv('resources/train.csv')
df.head()

Using TensorFlow backend.


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 56 columns):
Id                                    15120 non-null int64
Elevation                             15120 non-null int64
Aspect                                15120 non-null int64
Slope                                 15120 non-null int64
Horizontal_Distance_To_Hydrology      15120 non-null int64
Vertical_Distance_To_Hydrology        15120 non-null int64
Horizontal_Distance_To_Roadways       15120 non-null int64
Hillshade_9am                         15120 non-null int64
Hillshade_Noon                        15120 non-null int64
Hillshade_3pm                         15120 non-null int64
Horizontal_Distance_To_Fire_Points    15120 non-null int64
Wilderness_Area1                      15120 non-null int64
Wilderness_Area2                      15120 non-null int64
Wilderness_Area3                      15120 non-null int64
Wilderness_Area4                      15120 non-null int64
Soil_T

In [5]:
scaler = StandardScaler()

def preprocess(df, train=True):
    numerical_columns = ['Elevation', 'Aspect', 'Slope', "Horizontal_Distance_To_Hydrology",
                     'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                     'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
    numerical_features = df[numerical_columns]
    if train:
        numerical_features = pd.DataFrame(scaler.fit_transform(numerical_features), columns=numerical_columns)
    else:
        numerical_features = pd.DataFrame(scaler.transform(numerical_features), columns=numerical_columns)
    categorical_features = df.select(lambda col: col.startswith('Wilderness_Area') or col.startswith('Soil_Type'), axis=1).copy()
    train_features = pd.concat([numerical_features, categorical_features], axis = 1)
    if train:
        return (train_features, pd.get_dummies(df['Cover_Type']))
    return train_features

In [6]:
train_features, train_target = preprocess(df)

In [7]:
train_features.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,-0.367095,-0.959980,-1.597132,0.146639,-0.834074,-0.908681,0.271454,0.571653,0.281259,4.334805,...,0,0,0,0,0,0,0,0,0,0
1,-0.381461,-0.914559,-1.715424,-0.072337,-0.932054,-0.999246,0.238732,0.703225,0.346627,4.285710,...,0,0,0,0,0,0,0,0,0,0
2,0.130912,-0.160577,-0.887379,0.194243,0.227369,1.106379,0.696843,0.834797,-0.002005,4.191156,...,0,0,0,0,0,0,0,0,0,0
3,0.085421,-0.015231,0.177250,0.070474,1.092853,1.038455,0.827731,0.834797,-0.285268,4.272981,...,0,0,0,0,0,0,0,0,0,0
4,-0.369489,-1.014485,-1.715424,-0.353198,-0.850404,-0.998491,0.238732,0.659368,0.324838,4.237524,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train_target.head()

,1,2,3,4,5,6,7
0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0


In [9]:
train_features, train_target = train_features.values, train_target.values

In [11]:
x_train, x_validation, y_train, y_validation = train_test_split(train_features, train_target, test_size=0.15)

In [12]:
model = Sequential([
    Dense(256, input_shape=train_features.shape[1:], activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_target.shape[1], activation='softmax')
])

model.compile(optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])


model.fit(train_features, train_target, 
             batch_size=120, epochs=10000, verbose=1, 
             validation_data=(x_validation, y_validation), 
             callbacks=[
                ModelCheckpoint("forest.h5", save_best_only=True),
                EarlyStopping(patience=40)
             ])

score = model.evaluate(x_validation, y_validation)
print('\n\n')
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

Train on 15120 samples, validate on 2268 samples
Epoch 1/10000
15120/15120 [==============================] - 1s - loss: 1.9462 - acc: 0.1508 - val_loss: 1.9410 - val_acc: 0.1825
Epoch 2/10000
15120/15120 [==============================] - 0s - loss: 1.9460 - acc: 0.1505 - val_loss: 1.9398 - val_acc: 0.1918
Epoch 3/10000
15120/15120 [==============================] - 0s - loss: 1.9438 - acc: 0.1562 - val_loss: 1.9386 - val_acc: 0.2033
Epoch 4/10000
15120/15120 [==============================] - 0s - loss: 1.9439 - acc: 0.1575 - val_loss: 1.9373 - val_acc: 0.2222
Epoch 5/10000
15120/15120 [==============================] - 0s - loss: 1.9427 - acc: 0.1550 - val_loss: 1.9360 - val_acc: 0.2346
Epoch 6/10000
15120/15120 [==============================] - 0s - loss: 1.9424 - acc: 0.1607 - val_loss: 1.9347 - val_acc: 0.2566
Epoch 7/10000
15120/15120 [==============================] - 0s - loss: 1.9409 - acc: 0.1624 - val_loss: 1.9334 - val_acc: 0.2813
Epoch 8/10000
15120/15120 [==============

15120/15120 [==============================] - 0s - loss: 1.4231 - acc: 0.3637 - val_loss: 1.2921 - val_acc: 0.3933
Epoch 64/10000
15120/15120 [==============================] - 0s - loss: 1.4137 - acc: 0.3707 - val_loss: 1.2818 - val_acc: 0.3990
Epoch 65/10000
15120/15120 [==============================] - 0s - loss: 1.4012 - acc: 0.3740 - val_loss: 1.2717 - val_acc: 0.4017
Epoch 66/10000
15120/15120 [==============================] - 0s - loss: 1.3927 - acc: 0.3803 - val_loss: 1.2619 - val_acc: 0.4017
Epoch 67/10000
15120/15120 [==============================] - 0s - loss: 1.3831 - acc: 0.3868 - val_loss: 1.2524 - val_acc: 0.4087
Epoch 68/10000
15120/15120 [==============================] - 0s - loss: 1.3750 - acc: 0.3835 - val_loss: 1.2424 - val_acc: 0.4167
Epoch 69/10000
15120/15120 [==============================] - 0s - loss: 1.3717 - acc: 0.3899 - val_loss: 1.2335 - val_acc: 0.4255
Epoch 70/10000
15120/15120 [==============================] - 0s - loss: 1.3555 - acc: 0.3995 - va

15120/15120 [==============================] - 0s - loss: 1.0369 - acc: 0.5422 - val_loss: 0.8893 - val_acc: 0.6389
Epoch 126/10000
15120/15120 [==============================] - 0s - loss: 1.0374 - acc: 0.5407 - val_loss: 0.8859 - val_acc: 0.6429
Epoch 127/10000
15120/15120 [==============================] - 0s - loss: 1.0340 - acc: 0.5463 - val_loss: 0.8821 - val_acc: 0.6451
Epoch 128/10000
15120/15120 [==============================] - 0s - loss: 1.0313 - acc: 0.5454 - val_loss: 0.8787 - val_acc: 0.6468
Epoch 129/10000
15120/15120 [==============================] - 0s - loss: 1.0230 - acc: 0.5521 - val_loss: 0.8751 - val_acc: 0.6486
Epoch 130/10000
15120/15120 [==============================] - 0s - loss: 1.0224 - acc: 0.5528 - val_loss: 0.8722 - val_acc: 0.6473
Epoch 131/10000
15120/15120 [==============================] - 0s - loss: 1.0141 - acc: 0.5567 - val_loss: 0.8688 - val_acc: 0.6512
Epoch 132/10000
15120/15120 [==============================] - 0s - loss: 1.0178 - acc: 0.54

15120/15120 [==============================] - 0s - loss: 0.8772 - acc: 0.6232 - val_loss: 0.7533 - val_acc: 0.6865
Epoch 188/10000
15120/15120 [==============================] - 0s - loss: 0.8751 - acc: 0.6151 - val_loss: 0.7516 - val_acc: 0.6896
Epoch 189/10000
15120/15120 [==============================] - 0s - loss: 0.8734 - acc: 0.6214 - val_loss: 0.7496 - val_acc: 0.6869
Epoch 190/10000
15120/15120 [==============================] - 0s - loss: 0.8705 - acc: 0.6178 - val_loss: 0.7488 - val_acc: 0.6887
Epoch 191/10000
15120/15120 [==============================] - 0s - loss: 0.8666 - acc: 0.6204 - val_loss: 0.7481 - val_acc: 0.6856
Epoch 192/10000
15120/15120 [==============================] - 0s - loss: 0.8722 - acc: 0.6202 - val_loss: 0.7457 - val_acc: 0.6869
Epoch 193/10000
15120/15120 [==============================] - 0s - loss: 0.8689 - acc: 0.6237 - val_loss: 0.7448 - val_acc: 0.6852
Epoch 194/10000
15120/15120 [==============================] - 0s - loss: 0.8670 - acc: 0.61

15120/15120 [==============================] - 0s - loss: 0.7967 - acc: 0.6516 - val_loss: 0.6876 - val_acc: 0.7055
Epoch 250/10000
15120/15120 [==============================] - 0s - loss: 0.7908 - acc: 0.6566 - val_loss: 0.6874 - val_acc: 0.7055
Epoch 251/10000
15120/15120 [==============================] - 0s - loss: 0.7854 - acc: 0.6573 - val_loss: 0.6859 - val_acc: 0.7050
Epoch 252/10000
15120/15120 [==============================] - 0s - loss: 0.7882 - acc: 0.6583 - val_loss: 0.6845 - val_acc: 0.7068
Epoch 253/10000
15120/15120 [==============================] - 0s - loss: 0.7881 - acc: 0.6542 - val_loss: 0.6840 - val_acc: 0.7041
Epoch 254/10000
15120/15120 [==============================] - 0s - loss: 0.7870 - acc: 0.6522 - val_loss: 0.6840 - val_acc: 0.7055
Epoch 255/10000
15120/15120 [==============================] - 0s - loss: 0.7866 - acc: 0.6571 - val_loss: 0.6827 - val_acc: 0.7112
Epoch 256/10000
15120/15120 [==============================] - 0s - loss: 0.7854 - acc: 0.65

...

15120/15120 [==============================] - 0s - loss: 0.1582 - acc: 0.9472 - val_loss: 0.0736 - val_acc: 0.9780
Epoch 1725/10000
15120/15120 [==============================] - 0s - loss: 0.1206 - acc: 0.9604 - val_loss: 0.1247 - val_acc: 0.9537
Epoch 1726/10000
15120/15120 [==============================] - 0s - loss: 0.1307 - acc: 0.9548 - val_loss: 0.0835 - val_acc: 0.9757
Epoch 1727/10000
15120/15120 [==============================] - 0s - loss: 0.1354 - acc: 0.9545 - val_loss: 0.0965 - val_acc: 0.9691
Epoch 1728/10000
15120/15120 [==============================] - 0s - loss: 0.1470 - acc: 0.9512 - val_loss: 0.1026 - val_acc: 0.9678
Epoch 1729/10000
15120/15120 [==============================] - 0s - loss: 0.1201 - acc: 0.9618 - val_loss: 0.0958 - val_acc: 0.9709
Epoch 1730/10000
15120/15120 [==============================] - 0s - loss: 0.1304 - acc: 0.9575 - val_loss: 0.1024 - val_acc: 0.9678
Epoch 1731/10000
15120/15120 [==============================] - 0s - loss: 0.1507 - ac

15120/15120 [==============================] - 0s - loss: 0.1145 - acc: 0.9624 - val_loss: 0.0685 - val_acc: 0.9806
Epoch 1786/10000
15120/15120 [==============================] - 0s - loss: 0.1520 - acc: 0.9484 - val_loss: 0.0674 - val_acc: 0.9828
Epoch 1787/10000
15120/15120 [==============================] - 0s - loss: 0.1147 - acc: 0.9618 - val_loss: 0.0578 - val_acc: 0.9832
Epoch 1788/10000
15120/15120 [==============================] - 0s - loss: 0.1070 - acc: 0.9671 - val_loss: 0.0875 - val_acc: 0.9705
Epoch 1789/10000
15120/15120 [==============================] - 0s - loss: 0.1178 - acc: 0.9601 - val_loss: 0.0857 - val_acc: 0.9718
Epoch 1790/10000
15120/15120 [==============================] - 0s - loss: 0.1180 - acc: 0.9635 - val_loss: 0.0701 - val_acc: 0.9806
Epoch 1791/10000
15120/15120 [==============================] - 0s - loss: 0.1588 - acc: 0.9483 - val_loss: 0.0678 - val_acc: 0.9819
Epoch 1792/10000
15120/15120 [==============================] - 0s - loss: 0.1185 - ac

15120/15120 [==============================] - 0s - loss: 0.1272 - acc: 0.9580 - val_loss: 0.0562 - val_acc: 0.9828
Epoch 1847/10000
15120/15120 [==============================] - 0s - loss: 0.1031 - acc: 0.9674 - val_loss: 0.0635 - val_acc: 0.9824
Epoch 1848/10000
15120/15120 [==============================] - 0s - loss: 0.0919 - acc: 0.9700 - val_loss: 0.0541 - val_acc: 0.9832
Epoch 1849/10000
15120/15120 [==============================] - 0s - loss: 0.1133 - acc: 0.9622 - val_loss: 0.0640 - val_acc: 0.9788
Epoch 1850/10000
15120/15120 [==============================] - 0s - loss: 0.1145 - acc: 0.9635 - val_loss: 0.0619 - val_acc: 0.9802
Epoch 1851/10000
15120/15120 [==============================] - 0s - loss: 0.1073 - acc: 0.9648 - val_loss: 0.0538 - val_acc: 0.9837
Epoch 1852/10000
15120/15120 [==============================] - 0s - loss: 0.0959 - acc: 0.9694 - val_loss: 0.0588 - val_acc: 0.9819
Epoch 1853/10000
15120/15120 [==============================] - 0s - loss: 0.1365 - ac

15120/15120 [==============================] - 0s - loss: 0.1058 - acc: 0.9647 - val_loss: 0.0753 - val_acc: 0.9753
Epoch 1908/10000
15120/15120 [==============================] - 0s - loss: 0.1121 - acc: 0.9640 - val_loss: 0.0690 - val_acc: 0.9780
Epoch 1909/10000
15120/15120 [==============================] - 0s - loss: 0.0916 - acc: 0.9706 - val_loss: 0.0458 - val_acc: 0.9881
Epoch 1910/10000
15120/15120 [==============================] - 0s - loss: 0.1258 - acc: 0.9586 - val_loss: 0.0919 - val_acc: 0.9678
Epoch 1911/10000
15120/15120 [==============================] - 0s - loss: 0.0878 - acc: 0.9719 - val_loss: 0.0555 - val_acc: 0.9837
Epoch 1912/10000
15120/15120 [==============================] - 0s - loss: 0.1074 - acc: 0.9647 - val_loss: 0.0436 - val_acc: 0.9890
Epoch 1913/10000
15120/15120 [==============================] - 0s - loss: 0.1136 - acc: 0.9625 - val_loss: 0.0638 - val_acc: 0.9802
Epoch 1914/10000
15120/15120 [==============================] - 0s - loss: 0.1005 - ac

15120/15120 [==============================] - 0s - loss: 0.0788 - acc: 0.9769 - val_loss: 0.0571 - val_acc: 0.9810
Epoch 1969/10000
15120/15120 [==============================] - 0s - loss: 0.0882 - acc: 0.9709 - val_loss: 0.0838 - val_acc: 0.9718
Epoch 1970/10000
15120/15120 [==============================] - 0s - loss: 0.0997 - acc: 0.9690 - val_loss: 0.0353 - val_acc: 0.9903
Epoch 1971/10000
15120/15120 [==============================] - 0s - loss: 0.0836 - acc: 0.9744 - val_loss: 0.0444 - val_acc: 0.9872
Epoch 1972/10000
15120/15120 [==============================] - 0s - loss: 0.1071 - acc: 0.9667 - val_loss: 0.0494 - val_acc: 0.9846
Epoch 1973/10000
15120/15120 [==============================] - 0s - loss: 0.0802 - acc: 0.9746 - val_loss: 0.0466 - val_acc: 0.9868
Epoch 1974/10000
15120/15120 [==============================] - 0s - loss: 0.0891 - acc: 0.9708 - val_loss: 0.0432 - val_acc: 0.9854
Epoch 1975/10000
15120/15120 [==============================] - 0s - loss: 0.0793 - ac

In [13]:
def cover_type_to_int(arr):
    return np.argmax(arr, axis=1) + 1

# http://maxberggren.se/2017/06/18/deep-learning-vs-xgboost/
pd.crosstab(
    pd.Series(cover_type_to_int(y_validation), name='Actual'),
    pd.Series(cover_type_to_int(model.predict(x_validation)), name='Predicted'),
    margins=True
)

Predicted,1,2,3,4,5,6,7,All
Actual,,,,,,,,
1,355,1,0,0,0,0,5,361
2,11,314,0,0,1,0,0,326
3,0,0,286,5,0,5,0,296
4,0,0,0,327,0,0,0,327
5,0,1,0,0,336,0,0,337
6,0,0,2,1,0,299,0,302
7,0,0,0,0,0,0,319,319
All,366,316,288,333,337,304,324,2268


In [26]:
test_df = pd.read_csv('resources/test.csv')
test_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,15121,2680,354,14,0,0,2684,196,214,156,...,0,0,0,0,0,0,0,0,0,0
1,15122,2683,0,13,0,0,2654,201,216,152,...,0,0,0,0,0,0,0,0,0,0
2,15123,2713,16,15,0,0,2980,206,208,137,...,0,0,0,0,0,0,0,0,0,0
3,15124,2709,24,17,0,0,2950,208,201,125,...,0,0,0,0,0,0,0,0,0,0
4,15125,2706,29,19,0,0,2920,210,195,115,...,0,0,0,0,0,0,0,0,0,0


In [27]:
ids = test_df['Id'].copy()
test_df = preprocess(test_df, train=False)
predictions = pd.Series(cover_type_to_int(model.predict(test_df.values)), name='Cover_Type')

In [28]:
result = pd.concat([ids, predictions], axis=1)
result.head()

,Id,Cover_Type
0,15121,2
1,15122,1
2,15123,1
3,15124,1
4,15125,2


In [29]:
result.to_csv('predictions.csv', index=False)